In [6]:
from music21 import *
import json
from typing import Optional, Dict, Any, List

In [2]:
us = environment.UserSettings()
us['musescoreDirectPNGPath'] = r'C:\Program Files\MuseScore 3\bin\MuseScore3.exe'

In [5]:
path = 'data/bach_fugue_1-16_example.musicxml'
score = converter.parse(path)

In [ ]:
score.show('midi')

In [7]:
def get_tinynotation_octave(pitch):
    """
    Map music21 pitch to the octave notation in your rules.
    CC to BB = C1 to B2 (MIDI 24-47)
    C to B = C2 to B3 (MIDI 36-59) 
    c to b = C4 to B4 (MIDI 60-71)
    c' to b' = C5 to B5 (MIDI 72-83)
    """
    octave = pitch.octave
    step = pitch.step.lower()
    
    if octave <= 1:
        # CCC, CC range (C0-B1)
        return pitch.step.upper() * (3 - octave)
    elif octave == 2:
        # CC to BB (C2-B2)
        return pitch.step.upper() * 2
    elif octave == 3:
        # C to B (C3-B3)
        return pitch.step.upper()
    elif octave == 4:
        # c to b (C4-B4, middle C octave)
        return step
    else:
        # c' to b' and higher (C5+)
        return step + "'" * (octave - 4)


In [8]:
def voice_to_tinynotation_custom(part: stream.Part) -> str:
    """
    Converts a single music21 Part (voice) to custom tinynotation
    according to the user rules.
    """
    tn = []
    prev_duration = None
    
    for elem in part.recurse().notesAndRests:
        # Duration mapping
        dur_map = {
            'whole': '1',
            'half': '2',
            'quarter': '4',
            'eighth': '8',
            '16th': '16',
            '32nd': '32',
            '64th': '64'
        }
        dur_num = dur_map.get(elem.duration.type, '4')
        
        # Dot
        dot_str = '.' if elem.duration.dots > 0 else ''
        
        # Check if we need to specify duration
        full_duration = dur_num + dot_str
        if full_duration == prev_duration:
            duration_str = ''
        else:
            duration_str = full_duration
            prev_duration = full_duration
        
        # Tie
        tie_str = '~' if elem.tie and elem.tie.type == 'start' else ''
        
        if isinstance(elem, note.Rest):
            # Always specify duration for rests
            tn.append(f"r{dur_num}{dot_str}{tie_str}")
        elif isinstance(elem, note.Note):
            # Get base note name (just the letter)
            base_name = elem.pitch.step.lower()
            
            # Accidental
            acc = ''
            if elem.pitch.accidental:
                alter = elem.pitch.accidental.alter
                if alter == 1:
                    acc = '#'
                elif alter == -1:
                    acc = '-'
                elif alter == 0:
                    acc = 'n'
                elif alter == 2:
                    acc = '##'
                elif alter == -2:
                    acc = '--'
                
                # Editorial accidentals in parentheses
                if hasattr(elem.pitch.accidental, 'editorial') and elem.pitch.accidental.editorial:
                    acc = f"({acc})"
            
            # Octave designation
            octave_str = get_tinynotation_octave(elem.pitch)
            
            # The octave_str already contains the letter, so extract just the octave part
            # Find where the letter ends
            letter = elem.pitch.step.upper() if octave_str[0].isupper() else elem.pitch.step.lower()
            octave_suffix = octave_str[1:]  # Everything after the first letter
            
            tn.append(f"{base_name}{octave_suffix}{acc}{duration_str}{tie_str}")
    
    return " ".join(tn)

In [ ]:
def score_to_tinyscore(score: stream.Score) -> str:
    """ Separeted by voices into new lines """
    
    tinyscore = ""
    for id, voice in enumerate(score.parts):
        tn_str = voice_to_tinynotation_custom(voice)
        tinyscore += f"V{id}: {tn_str}\n"
    return tinyscore

In [9]:
score = converter.parse(path).explode()

tinyscore = ""
for id, voice in enumerate(score.parts):
    tn_str = voice_to_tinynotation_custom(voice)
    tinyscore += f"V{id}: {tn_str}\n"
print(tinyscore)

V0: r2 r8 g# b# g# c'# r8 b# r8 a#16 g# f#8~ f#16 g# a# f# g#8 r8 c'# r8 c'# r8 b# r8 r8 c'# e'# c'# g'# r8 f'# r8 e'#16 d'# c'#8~ c'#16 d'# e'# c'# r16 d'# f'# d'# g'#8 r8 f'# r8 e'#8. e'#16 f'##8 g'#~ g'#16 g'# f'##8 r1 r1 r1 r1 r1 r1 r1 r1 r1 r1
V1: r1 r8 g# e# g# c# r8 d# r8 e#16 f# g#8~ g#16 f# e# g# d# e# f#8~ f#16 e# d# f# e#8 r8 r4 r8 g# b# g# c'# r8 bn r8 a# r8 r16 a# c'## a# d'#8 r8 r16 d'# c'#8~ c'#16 b#8 b#16 a#8. c'#16 r1
V2: r8 c# e# c# g# r8 f# r8 e#16 d# c#8~ c#16 d# e# c# f#8 r8 fF# r8 r8 c# aA# c# fF# aA# dD# gG# cC# r8 r8 c# b# c# g# r8 r8 c# e# c# f# r8 e# r8 r16 d# bB# d# gG##8 aA# dD# eE# cC# dD# r1 r1 r1 r1 r1 r1 r1 r1 r1 r1



In [10]:
converter.parse(tinyscore.split('\n')[0], format='tinynotation').show('midi')

# Generating Labels

In [11]:
from prompts import LABEL_GENERATION_PROMPT

In [ ]:
prompt = LABEL_GENERATION_PROMPT.build_prompt(
    COMPOSER_NAME=...,
    NATIONALITY=...,
    STYLE=...,
    KEY=...,
    METER=...,
    NUMBER=...,
    PASTE_TINYNOTATION_HERE=
)

In [ ]:
print(prompt)


You are a symbolic music analysis assistant trained in Western tonal theory and Eastern European art music.
Your task is not to generate music, but to analyze short symbolic score excerpts and annotate high-level musical entities in a conservative, explainable manner.
When uncertain, you must say so explicitly.

## TINYANOTATION RULES
Here are the most important rules by default:

1. Note names are: a,b,c,d,e,f,g and r for rest
2. Flats, sharps, and naturals are notated as #,- (not b), and (if needed) n. If the accidental is above the staff (i.e., editorial), enclose it in parentheses: (#), etc. Make sure that flats in the key signatures are explicitly specified.
3. Note octaves are specified as follows:
    CC to BB = from C below bass clef to second-line B in bass clef
    C to B = from bass clef C to B below middle C.
    c to b = from middle C to the middle of treble clef
    c' to b' = from C in treble clef to B above treble clef
    Octaves below and above these are specified by

In [ ]:
import json

In [ ]:
def read_json(path: str) -> Any:
    with open(path, 'r') as f:
        return json.load(f)

In [ ]:
result = read_json('data/example_result.json')

In [ ]:
for res in result:
    if res.get("confidence") == 'high':
        print(json.dumps(res, indent=2)) 
        example = res.get("example", None)
        
        if example:
            if isinstance(example, dict):
                for voice, tn in example.items():
                    print(f"Voice: {voice}, TinyNotation: {tn}")
                    score = converter.parse(tn, format='tinynotation')
                    score.show('midi')
                    score.show('musicxml.png')  # saves and opens a PNG of the score
            else:
                score = converter.parse(example, format='tinynotation')
                score.show('musicxml.png')  # saves and opens a PNG of the score
                score.show('midi')

{
  "entity_type": "MOTIF",
  "start_bar": 1,
  "end_bar": 1,
  "voices": [
    "V0"
  ],
  "example": "r8 g# b# g# c#'",
  "confidence": "high",
  "justification": "A short, rhythmically distinct rising-falling figure that recurs in V0, forming a clear motivic idea."
}


{
  "entity_type": "PHRASE",
  "start_bar": 1,
  "end_bar": 3,
  "example": {
    "V0": "r2 r8 g# b# g# c#' r8 b# r8 a#16 g# f#8~ f#16 g# a# f# g#8",
    "V1": "r1 r8 g# e# g# c# r8 d# r8 e#16 f# g#8~ g#16 f# e# g# d# e# f#8~ f#16 e# d# f# e#8"
  },
  "confidence": "high",
  "justification": "Bars 1\u0432\u0402\u201c3 form a coherent musical unit across V0 and V1, ending in a partial pause."
}
Voice: V0, TinyNotation: r2 r8 g# b# g# c#' r8 b# r8 a#16 g# f#8~ f#16 g# a# f# g#8


Voice: V1, TinyNotation: r1 r8 g# e# g# c# r8 d# r8 e#16 f# g#8~ g#16 f# e# g# d# e# f#8~ f#16 e# d# f# e#8


{
  "entity_type": "MOTIF",
  "start_bar": 4,
  "end_bar": 5,
  "voices": [
    "V2"
  ],
  "example": "r8 c# e# c# g#",
  "confidence": "high",
  "justification": "Short, repeated figure in V2 is rhythmically compact and clearly recurring."
}


In [ ]:
def highlight_entity(part, start_bar, end_bar, color="red"):
    # Use getElementsByClass(Measure) to get measures
    for m in part.getElementsByClass('Measure'):
        if start_bar <= m.number <= end_bar:
            for n in m:  # notes only
                n.style.color = color

In [ ]:
entity_colors = {
    "THEME": "red",
    "MOTIF": "blue",
    "SEQUENCE": "green",
    "CADENCE": "purple",
    "MODAL_HINT": "orange"
}

score = converter.parse("data/bach_fugue_1-16_example.musicxml")
score = score.explode()
parts = score.parts

for res in result:
    color = entity_colors.get(res['entity_type'], "black")
    example = res.get("example", None)
    if example and isinstance(example, dict):
        for voice, tn in example.items():
            part_idx = int(voice[1])  # V0 → 0, V1 → 1
            start, end = res['start_bar'], res['end_bar']
            highlight_entity(parts[part_idx], start, end, color=color)


In [ ]:
from music21 import stream, instrument

score_show = stream.Score()

for i, p in enumerate(parts):
    # Ensure p is a Part
    if not isinstance(p, stream.Part):
        part = stream.Part()
        part.append(p)
    else:
        part = p

    # Give it a proper ID so order is preserved
    part.id = f"V{i}"
    # Optional: assign instrument to control display
    part.insert(0, instrument.fromString(f"Voice {i}"))
    
    score_show.append(part)

score_show.show('musicxml.png')  # saves and opens a PNG of the score